In [12]:
import ee
import pandas as pd
from datetime import datetime, timezone

# Initialize Earth Engine
ee.Initialize(project='data690-zhouhaomatt')

# Define the central point in WGS84 (latitude, longitude)
central_point = ee.Geometry.Point([170.89170634746554, -43.999047918238475])
# Define the grid size and pixel size (in meters, Sentinel-2 resolution)
grid_size = 5
pixel_size = 10  # Sentinel-2 resolution is 10 meters per pixel

# Function to calculate grid cell centers
def calculate_grid_points(central_point, pixel_size, grid_size):
    central_coords = central_point.coordinates().getInfo()
    grid_points = []
    for dy in range(-2, 3):
        for dx in range(-2, 3):
            x = central_coords[0] + dx * pixel_size * 0.00001  # Convert meters to degrees (approx)
            y = central_coords[1] + dy * pixel_size * 0.00001  # Convert meters to degrees (approx)
            grid_points.append(ee.Geometry.Point([x, y]))
    return grid_points

# Get the grid points
grid_points = calculate_grid_points(central_point, pixel_size, grid_size)

# Indices for the specific pixels to form a "C" shape downwards
top_row_indices = [20, 21, 22, 23, 24]  # First row: all 5 pixels
second_row_indices = [15, 19]  # Second row: first and last pixels
third_row_indices = [10, 14]  # Third row: first and last pixels

# Combine all selected points
selected_indices = top_row_indices + second_row_indices + third_row_indices 
selected_points = [grid_points[i] for i in selected_indices]

# Convert to ee.List and create a FeatureCollection
selected_points_fc = ee.FeatureCollection([ee.Feature(p) for p in selected_points])

# Load Sentinel-2 image collection and filter by date, region, and cloud coverage
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2023-02-18', '2024-05-31') \
    .filterBounds(central_point) 

# Select relevant bands
bands = ['B2', 'B3', 'B4', 'B8']  # (Blue, Green, Red, NIR)

# Define weights for the aggregation
weights = {
    0: 0.15, 1: 0.15, 2: 0.15, 3: 0.15, 4: 0.15,  # Top row
    5: 0.075, 6: 0.075,  # Second row
    7: 0.05, 8: 0.05  # Third row
}

# Function to sample the image and calculate weighted averages
def sample_image(image):
    sampled_values = image.select(bands).sampleRegions(
        collection=selected_points_fc,
        scale=pixel_size,
        geometries=True
    ).getInfo()
    timestamp = image.get('system:time_start').getInfo() / 1000
    date = datetime.fromtimestamp(timestamp, timezone.utc).strftime('%Y-%m-%d')
    data = {'timestamp': date}
    for band in bands:
        weighted_sum = sum(sampled_values['features'][i]['properties'][band] * weights[i] for i in range(9))
        data[band] = weighted_sum
    return data

# Process all images and store the results
results = []
for image in sentinel2.toList(sentinel2.size()).getInfo():
    img = ee.Image(image['id'])
    result = sample_image(img)
    results.append(result)

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df.to_csv('sentinel2_aggregated.csv', index=False)

print("Data processing complete. Sampled data saved to 'sentinel2_aggregated.csv'.")


Data processing complete. Sampled data saved to 'sentinel2_aggregated.csv'.


In [13]:
df

,timestamp,B2,B3,B4,B8
0,2023-02-20,1687.850,1483.125,1284.700,1218.125
1,2023-02-22,6124.250,5805.900,6061.875,6799.550
2,2023-02-25,1741.525,1559.825,1328.000,1248.925
3,2023-02-27,1830.575,1596.450,1351.025,1259.525
4,2023-03-02,1823.475,1618.100,1371.175,1333.125
...,...,...,...,...,...
176,2024-05-17,6478.250,5955.400,6264.900,6866.200
177,2024-05-20,4514.250,4005.850,4100.900,4295.575
178,2024-05-22,2097.550,1753.825,1471.700,1272.625
179,2024-05-25,2075.700,1788.050,1500.275,1268.300


In [14]:
env = pd.read_csv('clean_data_daily.csv')

In [15]:
merged_full_df = pd.merge(df, env, on='timestamp', how='inner')


In [2]:
merged_full_df

NameError: name 'merged_full_df' is not defined

In [1]:
merged_full_df.to_csv('merged_data_full.csv')

NameError: name 'merged_full_df' is not defined

In [7]:
import ee
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Initialize the Earth Engine module.
ee.Initialize(project='data690-zhouhaomatt')

# Define the area of interest (AOI).
point = ee.Geometry.Point([170.89165309, -43.99906932])

# Define the time period for analysis.
start_date = '2018-01-01'
end_date = '2024-05-31'

# Load the harmonized Sentinel-2 surface reflectance collection.
collection_harmonized = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(point) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .select('B8')

# Function to map over the collection and extract NIR values.
def get_nir_value(image):
    # Reduce the region to get the mean NIR value at the point.
    nir_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=10
    ).get('B8')
    
    # Create a feature with the date and NIR value.
    return ee.Feature(None, {
        'date': image.date().format('YYYY-MM-dd'),
        'nir': nir_value
    })

# Apply the function to the collection.
nir_features_harmonized = collection_harmonized.map(get_nir_value).filter(ee.Filter.notNull(['nir']))

# Convert the features to a list and then to a pandas DataFrame.
def features_to_dataframe(feature_collection):
    features = feature_collection.getInfo()['features']
    data = [{'date': f['properties']['date'], 'nir': f['properties']['nir']} for f in features]
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    df.rename(columns={'date': 'timestamp'}, inplace=True)
    return df

nir_values_harmonized = features_to_dataframe(nir_features_harmonized)

# Print the DataFrame to check
print(nir_values_harmonized.head())


   timestamp  nir
0 2018-12-23  187
1 2018-12-28   49
2 2018-12-30  198
3 2019-01-02  237
4 2019-01-04    1


In [8]:
nir_values_harmonized.to_csv('remotesensing2.csv')